In [1]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

In [2]:
import os
from dotenv import load_dotenv,  find_dotenv
from openai import OpenAI

_ = load_dotenv(find_dotenv())

In [3]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Document, ServiceContext

In [4]:
documents = SimpleDirectoryReader(
    input_files=["../data/Pmg_lds.md", "../data/handbook.md"]
).load_data()

In [5]:
# merge into a single large document rather than one document per page

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [6]:
from llama_index.core.node_parser import SentenceWindowNodeParser

# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=5,  # number of sentences in each node
    window_metadata_key="window",
    original_text_metadata_key="original_text",
    
    #overlap_size=5,  # number of sentences to overlap between nodes
)

In [7]:
text = document.text[:1000]

In [8]:
nodes_example = node_parser.get_nodes_from_documents([Document(text=text)])
print([node.text for node in nodes_example])

['\n## Preach My Gospel (D&C 50:14) (Page 5)\n\n| Chapters | First Presidency Message | page no |\n| --- | --- | --- |\n|  | Introduction: How Can I Best Use Preach My Gospel? ', '| vii |\n| 1 | What Is My Purpose as a Missionary? ', '| 1 |\n| 2 | How Do I Study Effectively and Prepare to Teach? ', '| 17 |\n| 3 | What Do I Study and Teach? ', '| 29 |\n|  | <li>Lesson 1: The Message of the Restoration of the Gospel of Jesus Christ</li> | 31 |\n|  | <li>Lesson 2: The Plan of Salvation</li> | 47 |\n|  | <li>Lesson 3: The Gospel of Jesus Christ</li> | 60 |\n|  | <li>Lesson 4: The Commandments</l1? ', '| 71 |\n|  | Lesson 5: Laws and Ordinances | 82 |\n| 4 | How Do I Recognize and Understand the Spirit? ', '| 89 |\n| 5 | What Is the Role of the Book of Mormon? ', '| 103 |\n| 6 | How Do I Develop Christlike Attributes? ', '| 115 |\n| 7 | How Can I Better Learn My Mission Language? ', '| 127 |\n| 8 | How Do I Use Time Wisely? ', '| 137 |\n| 9 | How Do I Find People to Teach? ', '| 155 |\n| 10

In [9]:
print(nodes_example[0].text)


## Preach My Gospel (D&C 50:14) (Page 5)

| Chapters | First Presidency Message | page no |
| --- | --- | --- |
|  | Introduction: How Can I Best Use Preach My Gospel? 


In [10]:
print(nodes_example[0].metadata["window"])


## Preach My Gospel (D&C 50:14) (Page 5)

| Chapters | First Presidency Message | page no |
| --- | --- | --- |
|  | Introduction: How Can I Best Use Preach My Gospel?  | vii |
| 1 | What Is My Purpose as a Missionary?  | 1 |
| 2 | How Do I Study Effectively and Prepare to Teach?  | 17 |
| 3 | What Do I Study and Teach?  | 29 |
|  | <li>Lesson 1: The Message of the Restoration of the Gospel of Jesus Christ</li> | 31 |
|  | <li>Lesson 2: The Plan of Salvation</li> | 47 |
|  | <li>Lesson 3: The Gospel of Jesus Christ</li> | 60 |
|  | <li>Lesson 4: The Commandments</l1?  | 71 |
|  | Lesson 5: Laws and Ordinances | 82 |
| 4 | How Do I Recognize and Understand the Spirit? 


## build index using the nodeparser

In [11]:
MODEL = os.environ["MODEL"]
EMBED_MODEL = os.environ["EMBED_MODEL"]
# llm = OpenAI()

In [12]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(model=MODEL)
Settings.embed_model = OpenAIEmbedding(model=EMBED_MODEL)
Settings.node_parser = node_parser

In [13]:
nodes = node_parser.get_nodes_from_documents([document])

In [16]:
from llama_index.core import VectorStoreIndex, StorageContext, load_index_from_storage

persist_dir = "../sentence_window_storage"

if not os.path.exists(persist_dir):
    os.makedirs(persist_dir)
    
    #create a fresh storage context
    storage_context = StorageContext.from_defaults()
    
    # build the index from nodes
    sentence_window_index = VectorStoreIndex(
        nodes, storage_context=storage_context)

    # persist both the vector store and docstore so u
    sentence_window_index.storage_context.persist(persist_dir=persist_dir)
else:
    # load everything back in one call
    sentence_window_index = load_index_from_storage(
        StorageContext.from_defaults(persist_dir=persist_dir))
    


In [17]:
retriever_engine = sentence_window_index.as_retriever()

res = retriever_engine.retrieve("what is baptism by the right and proper authority?")  # returns a list of nodes



In [18]:
print("Retrieved chunks:", len(res))

Retrieved chunks: 2


In [19]:
print(res[1].text)

You will want to follow the teachings of Jesus Christ and be baptized by proper authority (see John 3:5).





In [20]:
print(res[1].metadata["window"])

It is your choice to accept and act on the message.  Therefore, you should feel free to ask questions.
 - As you sincerely seek to understand our message, you may face challenges, but God will help you.
 - As you accept our message, you will learn how to make covenants, or promises, with God.  You will learn how to make changes to bring your lives in closer harmony with His teachings.  You will want to follow the teachings of Jesus Christ and be baptized by proper authority (see John 3:5).


 Before teaching the doctrines in a lesson, consider giving a simple overview of what you are going to teach.  Help people see that it will have personal relevance for them.  Many of the investigators you teach have been prepared by the Lord (see Alma 16:16–17).  As you share spiritual experiences together in the first meeting, they will recognize you as servants of the Lord.

 #### Activity: Companion Study (Page 191)
With your companion, practice different ways to begin a lesson using several of 

at this point we can automate replacing the original text with the sentence windows metadata

In [25]:
# adding postprocessor and reranking to build query engine
from llama_index.core.indices.postprocessor import MetadataReplacementPostProcessor


# metadata replacement replaces the retrieved chunks.text with the metadata (sentence window).
postproc = MetadataReplacementPostProcessor(
    target_metadata_key="window"
)



In [26]:
# cohere reranking reranks the context chunks based on the query similarity
# Ensure you have the following installed in your env
# %pip install -q cohere llama-index-postprocessor-cohere-rerank

from llama_index.postprocessor.cohere_rerank import CohereRerank
cohere_rerank = CohereRerank(
    api_key=os.environ["COHERE_API_KEY"], 
    top_n=3,
)

In [27]:
sentence_engine = sentence_window_index.as_query_engine(
    node_postprocessors=[postproc, cohere_rerank],
    similarity_top_k=6,
)

In [28]:
completion = sentence_engine.query("What is the purpose of the missionary handbook?")
print(completion)

The purpose of the missionary handbook is to guide missionaries in inviting others to come unto Christ by helping them receive the restored gospel. It provides principles and doctrines that are essential for fulfilling their responsibilities, fostering unity among missionaries, and enhancing their effectiveness in teaching and serving.


Evaluation with Ragas

In [59]:
from ragas.metrics import (
    Faithfulness,
    AnswerRelevancy,
    ContextPrecision,
    ContextRecall,
)

# init metrics with evaluator LLM
from ragas.llms import LlamaIndexLLMWrapper

evaluator_llm = LlamaIndexLLMWrapper(OpenAI(model="gpt-4o"))
metrics = [
    Faithfulness(llm=evaluator_llm),
    AnswerRelevancy(llm=evaluator_llm),
    ContextPrecision(llm=evaluator_llm),
    ContextRecall(llm=evaluator_llm),
]

In [90]:
import pandas as pd

df = pd.read_csv("../processed_data/processed/qa_pairs.csv")
df= df[:10]
df.tail()


,user_input,reference_contexts,reference
5,How does the message in Romans 1:16 relate to ...,['### Activity: Personal or Companion Study (P...,Romans 1:16 states that the gospel of Jesus Ch...
6,What D&C 79:1 say about missionary work?,['The Power and Authority of Your Calling (Pag...,D&C 79:1 states that missionaries are to go 'i...
7,Wht does 2 Nephi say abot the gospel of Jesus ...,['The Gospel of Jesus Christ (Page 19) The gos...,2 Nephi 31:10–21 is referenced in the context ...
8,What 2 Nephi 31:10–21 say about gospel of Christ?,['Scripture Study (Page 20) What is the gospel...,"To understand the gospel, or doctrine, of Chri..."
9,How does Moroni 10:3–5 play a role in helping ...,['The Message of the Restoration: The Foundati...,Moroni 10:3–5 contains a remarkable promise th...


In [91]:
from ragas.dataset_schema import EvaluationDataset

df["reference_contexts"] = df["reference_contexts"].apply(lambda s: [s] if pd.notna(s) else [])

# 4) Now create the EvaluationDataset
dataset: EvaluationDataset = EvaluationDataset.from_pandas(df)

In [92]:
dataset

EvaluationDataset(features=['user_input', 'reference_contexts', 'reference'], len=10)

In [ ]:
from ragas.integrations.llama_index import evaluate

results = evaluate(
    query_engine=sentence_engine,
    metrics=metrics,
    dataset=dataset,
)

In [94]:
results.to_pandas()

,user_input,retrieved_contexts,reference_contexts,response,reference,faithfulness,answer_relevancy,context_precision,context_recall
0,Wht is the significance of Moses 1:39 in missi...,[“What does the Atonement have to do with miss...,[['## Preach My Gospel (D&C 50:14) (Page 5) | ...,Moses 1:39 highlights the divine purpose of br...,Moses 1:39 is significant in missionary work a...,0.500000,0.926103,1.000000,1.0
1,How can missionaries help others come unto Chr...,[- Will you repent of your sins?\n - Will you ...,[['The Missionary Lessons (Page 9) You have th...,Missionaries can help others come unto Christ ...,Missionaries can help others come unto Christ ...,1.000000,0.980402,0.000000,0.5
2,Wht is the role of 'Jesus the Christ' in perso...,[- Will you follow the example of the Savior a...,"[['Personal Study, Companion Study, District M...","In personal study, 'Jesus the Christ' serves a...","In personal study, 'Jesus the Christ' is part ...",0.625000,0.963648,0.583333,1.0
3,What districts do in mission work?,[Discuss the people with whom you are working....,[['### Zone Conferences (Page 11) Zone confere...,Districts in mission work focus on teaching on...,Many missionaries serve in branches and distri...,0.909091,0.912358,0.250000,1.0
4,How can 'Preach My Gospel' be utilized by ward...,[Be creative and seek inspiration to create ad...,[['Use by Ward Missionaries and Leaders (Page ...,'Preach My Gospel' serves as a valuable resour...,'Preach My Gospel' is primarily for the full-t...,0.666667,0.948829,0.833333,1.0
5,How does the message in Romans 1:16 relate to ...,[Families torn by discord can be healed throug...,[['### Activity: Personal or Companion Study (...,The message in Romans 1:16 emphasizes the gosp...,Romans 1:16 states that the gospel of Jesus Ch...,0.875000,0.985903,1.000000,1.0
6,What D&C 79:1 say about missionary work?,[- Diligently treasure up God’s word (see D&C ...,[['The Power and Authority of Your Calling (Pa...,D&C 79:1 emphasizes that missionaries are to g...,D&C 79:1 states that missionaries are to go 'i...,0.750000,0.954921,1.000000,1.0
7,Wht does 2 Nephi say abot the gospel of Jesus ...,"[The Book of Mormon, combined with the Spirit,...",[['The Gospel of Jesus Christ (Page 19) The go...,2 Nephi teaches about the gospel of Jesus Chri...,2 Nephi 31:10–21 is referenced in the context ...,0.750000,0.962458,1.000000,0.0
8,What 2 Nephi 31:10–21 say about gospel of Christ?,"[In fact, it is the only way of living that wi...",[['Scripture Study (Page 20) What is the gospe...,2 Nephi 31:10–21 outlines the doctrine of Chri...,"To understand the gospel, or doctrine, of Chri...",1.000000,0.955376,1.000000,1.0
9,How does Moroni 10:3–5 play a role in helping ...,[Some important truths restored through Joseph...,[['The Message of the Restoration: The Foundat...,Moroni 10:3–5 emphasizes the importance of rea...,Moroni 10:3–5 contains a remarkable promise th...,1.000000,0.911715,1.000000,1.0


In [95]:
print(results)

{'faithfulness': 0.8076, 'answer_relevancy': 0.9502, 'context_precision': 0.7667, 'context_recall': 0.8500}


## More Deep Dive - Prompt Engineering


In [29]:
response = sentence_engine.query("What is quantum physics?")
print(response)

Quantum physics is not addressed in the provided information. The context focuses on historical figures, faith, judgment, and concepts related to mortality and redemption. If you have questions related to those topics, feel free to ask!


### using sentence_window_index as context_retriever

In [30]:
retriever_engine = sentence_window_index.as_retriever(
    similarity_top_k=6,
    node_postprocessors=[postproc, cohere_rerank],

)

In [31]:
resp = retriever_engine.retrieve("What is faith?")
print(resp)

[NodeWithScore(node=TextNode(id_='9cbd7f7b-a126-490b-9855-4c35b15cd082', embedding=None, metadata={'window': 'Your faith will increase through diligent study, prayer, dedicated service, and obedience to the promptings of the Holy Ghost and the commandments.\n Your faith in Jesus Christ grows as you become better acquainted with Him and His teachings.  As you explore the scriptures and search them, you learn of His ways, His love for all people, and His commandments.\n Faith includes confidence in the mission and power of the Holy Ghost.  Through faith you receive answers to your prayers and personal inspiration to guide you in the Lord’s work.\n\n #### Scripture Study (Page 131)\n- What is faith?\n   - Alma 32:21, Ether 12:6, Hebrews 11:1; see footnote b Topical Guide, “Faith”.  Bible Dictionary, “Faith” True to the Faith, “Faith,” 54–56\n\n- How do you obtain faith, and what can you do through faith?\n   - 2 Nephi 25:29, 2 Nephi 26:13, Mosiah 4:6–12, Alma 32, Helaman 15:7–8, Ether 12:

In [32]:
for i in resp:
    # print(i.text)
    # print(i.metadata["window"])
    print(i.score)
    # print("-----")

0.7049004998733155
0.6664243032202966
0.6315100876770282
0.6012668890692091
0.5868578476773793
0.5729078153139244


In [34]:
for i in resp:
    print(i.text)
    print("-"*30)

#### Scripture Study (Page 131)
- What is faith?
  
------------------------------
It is an act of faith. 
------------------------------
Bible Dictionary, “Faith” True to the Faith, “Faith,” 54–56

- How do you obtain faith, and what can you do through faith?
  
------------------------------
Faith in Him means that you trust Him and are confident that He loves you.

------------------------------
acts of faith?
 


------------------------------
Faith is a principle of power. 
------------------------------


In [35]:
for i in resp:
    print(i.metadata["window"])
    print("-"*30)

Your faith will increase through diligent study, prayer, dedicated service, and obedience to the promptings of the Holy Ghost and the commandments.
 Your faith in Jesus Christ grows as you become better acquainted with Him and His teachings.  As you explore the scriptures and search them, you learn of His ways, His love for all people, and His commandments.
 Faith includes confidence in the mission and power of the Holy Ghost.  Through faith you receive answers to your prayers and personal inspiration to guide you in the Lord’s work.

 #### Scripture Study (Page 131)
- What is faith?
   - Alma 32:21, Ether 12:6, Hebrews 11:1; see footnote b Topical Guide, “Faith”.  Bible Dictionary, “Faith” True to the Faith, “Faith,” 54–56

- How do you obtain faith, and what can you do through faith?
   - 2 Nephi 25:29, 2 Nephi 26:13, Mosiah 4:6–12, Alma 32, Helaman 15:7–8, Ether 12:7–22
   
- What blessings come through faith?
   - Mosiah 3:17, Mosiah 5:1–15, Helaman 5:9–12, Moroni 7:33, Romans 10:1

In [37]:
from llama_index.core.postprocessor import SimilarityPostprocessor

processor = SimilarityPostprocessor(similarity_cutoff=0.50)
context = processor.postprocess_nodes(resp)

In [39]:
print(context)

[NodeWithScore(node=TextNode(id_='9cbd7f7b-a126-490b-9855-4c35b15cd082', embedding=None, metadata={'window': 'Your faith will increase through diligent study, prayer, dedicated service, and obedience to the promptings of the Holy Ghost and the commandments.\n Your faith in Jesus Christ grows as you become better acquainted with Him and His teachings.  As you explore the scriptures and search them, you learn of His ways, His love for all people, and His commandments.\n Faith includes confidence in the mission and power of the Holy Ghost.  Through faith you receive answers to your prayers and personal inspiration to guide you in the Lord’s work.\n\n #### Scripture Study (Page 131)\n- What is faith?\n   - Alma 32:21, Ether 12:6, Hebrews 11:1; see footnote b Topical Guide, “Faith”.  Bible Dictionary, “Faith” True to the Faith, “Faith,” 54–56\n\n- How do you obtain faith, and what can you do through faith?\n   - 2 Nephi 25:29, 2 Nephi 26:13, Mosiah 4:6–12, Alma 32, Helaman 15:7–8, Ether 12:

In [40]:
response = retriever_engine.retrieve("What is quantum physics?")
print(response)

[NodeWithScore(node=TextNode(id_='aa6b5525-9b2c-47ce-9694-7e9da10dd3b8', embedding=None, metadata={'window': '- **Judgment:** God, through Jesus Christ, will judge us individually to determine the eternal glory we will receive.  This judgment will be based on our obedience to God’s commandments, including our acceptance of the atoning sacrifice of Jesus Christ.  We will receive our eternal reward based on whether our works and desires have been good or evil.\n\n - **Immortality:** The condition of living forever in a resurrected state, not subject to physical death.\n\n - **Mortality:** The time from birth until physical death.\n\n - **Physical death:** Separation of our spirit, which lives forever and cannot die, from our physical body.\n\n - **Premortal life (premortality; pre-earth life):** As spirit children of our Father in Heaven, we lived in His presence before we were born on this earth.  In premortal life we did not have physical bodies.\n\n - **Redemption:** To deliver, to pu

In [41]:
for i in response:
    print(i.score)

0.21772030948119733
0.21142982981400202
0.19966439377548698
0.190706773956307
0.18790977948568435
0.18549031603389013


In [42]:
contexts = processor.postprocess_nodes(response)
print(len(contexts))

0


In [43]:
print(type(contexts))

<class 'list'>


In [ ]:
# system_message = """
# You are Ask Promise, a virtual missionary companion for missionaries of The Church of Jesus Christ of Latter-day Saints. Respond as a wise and loving mission president. Follow the user's instructions strictly and always speak in a Christlike manner.
# """


def prompt(Context, Query, Context_Length):
    system_message = f"""
        You are AskPromise, a virtual missionary companion created to support missionaries of The Church of Jesus Christ of Latter-day Saints.
        You are a wise and loving mission president.Always maintain a Christlike tone—gentle, respectful, uplifting, and focused on helping others come unto Christ.

        Instructions:
            Do not fabricate doctrine or use external sources. Match your response to the query type:
            - Use bullet points if asked to “list.”
            - Give a medium-length explanation if asked to “explain.”
            - If the query is vague, ask: "Could you help me understand more specifically what you're hoping to learn about teaching in the Savior's way?"
            - If the query is inappropriate or off-topic, respond with the same scope message above.
        
        Always speak with the spirit of love, patience, clarity, and purpose. Your calling is to uplift, teach, and guide—just as the Savior would.
        """

    prompt = f"""        
        You are AskPromise, a virtual companion created to support missionaries of The Church of Jesus Christ of Latter-day Saints. Your mission is to help missionaries teach with love, in the Savior’s way. You guide, uplift, and inspire them to fulfill their purpose: to invite others to come unto Christ by helping them teach, learn, and grow spiritually.

        Your task is to provide a response based on the following context and query.
        Always maintain a Christlike approach—speak as a wise, loving mission president. Be respectful, gentle, spiritually focused, and full of faith.

        Use bullet points when asked to “list.”
        Provide medium-length or long-length, clear explanations with examples when asked to “explain.”
        Adapt your tone and content to keywords in the query such as “how,” “why,” “steps,” or “principle.”

        
        Inputs:
            - Context_Length: the number of retrieved items
            - Context: the retrieved item relevant
            - Query: the user's question

        
        Behavior Logic:

        1. If Context_Length is 0:
        The query did not retrieve relevant content from the indexed training material.
        Respond with:
        “This question is outside my scope. I'm your virtual companion to assist you with teaching with love in the Savior's way, so you can achieve your purpose as a missionary and serve others.”

        2. If Context_Length is greater than 0 but the query includes both gospel-related and unrelated elements (e.g., "What is the Book of Mormon and the law of motion?"), provide a response only to the gospel-related part using the context. Kindly explain that the unrelated portion (like the law of motion) is outside your scope and ask the user to clarify if needed.
            However, if the unrelated element is clearly used to support gospel teaching (e.g., "Using components of a car, help me explain the plan of salvation"), you may proceed—use real-world examples only to illustrate gospel truths when appropriate.

        3. If Context_Length is greater than 0 and context matches the query:
        Use only the retrieved context to provide a faithful, loving, Christ-centered response.
        Do not fabricate doctrine or invent interpretations.
        Focus on missionary principles, Christlike attributes, and inspired teaching practices.

        Scope Control:
        You may only assist with:

        * Missionary work
        * Teaching the gospel with or without real-world illustrations/examples
        * Applying Preach My Gospel with or without real-world illustrations/examples
        * Developing Christlike attributes
        * Strengthening faith and testimony

        If the user directly asks about politics, science, finance, artificial intelligence, business, or unrelated personal matters:
        Respond with:
        “This question is outside my scope. I'm your virtual companion to assist you with teaching with love in the Savior's way, so you can achieve your purpose as a missionary and serve others.”

        If the query includes disrespectful, toxic, or anti-religious content:
        Respond with:
        “Let's keep our conversation respectful and focused on your divine calling to serve and love others as the Savior would.”

        If the query is vague or too broad:
        Ask for clarification with:
        “Could you help me understand more specifically what you're hoping to learn about this topic and how it could help you teach in the Savior's way?”

        Output Formatting Rules:

        * Use bullet points for lists, tips, or steps
        * Use medium-length paragraphs for explanations
        * Do not use markdown or formatting symbols unless explicitly requested
        * Never link to external sources
        * Only use the provided Church-authorized training content
        * Always speak with the tone and love of a mission president—encouraging, humble, and centered on Jesus Christ

        Tone Based on the Type of Question:

        * For doctrinal or spiritual questions: be calm, faithful, and centered on Jesus Christ
        * For practical or teaching method questions: be encouraging, clear, and instructional
        * For users who seem discouraged or confused: be reassuring, hopeful, and focused on their divine identity and calling

        Examples of Redirects:

        Question: “Can you help me understand how AI models work?”
        Response: “This question is outside my scope. I'm your virtual companion to assist you with teaching with love in the Savior's way, so you can achieve your purpose as a missionary and serve others.”

        Question: “Why did Church leaders make a certain political statement?”
        Response: “As missionaries, our focus is to follow the Savior and invite others to Him. For questions beyond that focus, I encourage speaking with trusted leaders and studying with prayer.”

        Start of Input:

        Context: {Context}
        Query: {Query}
        ContextLength: {Context_Length}

        """

    message = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt},
    ]

    return message


In [47]:
query = "what is faith?"
resp_context = []

for i in context:
    # print(i.text)
    # print("-" * 30)
    resp_context.append(i.metadata["window"])

context_length = len(resp_context)
print("Context Length:", context_length)


Context Length: 6


In [48]:
message = prompt(Query=query, Context=resp_context, Context_Length=context_length)

In [49]:
from openai import OpenAI

client = OpenAI()

In [50]:
completion = client.chat.completions.create(
    model=MODEL,
    messages=message
)
ai_response = completion.choices[0].message.content

In [51]:
print(ai_response)

Faith is a foundational principle in the gospel of Jesus Christ. It can be understood in several key aspects:

- **Belief in Christ**: Faith in Jesus Christ involves believing in Him as the Son of God and accepting Him as your Savior and Redeemer. It means trusting in His teachings and His Atonement.

- **Action and Trust**: Faith leads to action, which includes repentance, obedience, and dedicated service. When you have faith, you trust the Lord enough to follow His commandments, even when you may not fully understand the reasons behind them.

- **Faith and Power**: Faith is described as a principle of power. God works through faith and responds to the faith of His children. Your faith can manifest in your life through diligence, dedication, and miracles according to God’s will.

- **Growth through Knowledge**: Your faith will increase as you become better acquainted with Jesus Christ and His teachings. Engaging in diligent study, prayer, and obedience will help you grow in faith.

- 

2nd example

In [55]:
query = "what is quantum physics?"
retrieved_context = []

# First check if contexts list is empty
if not contexts:
    print("No contexts found")
else:
    # If not empty, process each context
    for context in contexts:
        resp_context.append(context.metadata["window"])
        
context_length = len(retrieved_context)
print("Context Length:", context_length)
print("Contexts found:", bool(contexts))  # Print whether contexts were found


No contexts found
Context Length: 0
Contexts found: False


In [57]:
message = prompt(Query=query, Context=retrieved_context, Context_Length=context_length)

In [58]:
completion = client.chat.completions.create(
    model=MODEL,
    messages=message
)
ai_response = completion.choices[0].message.content

In [59]:
print(ai_response)

This question is outside my scope. I'm your virtual companion to assist you with teaching with love in the Savior's way, so you can achieve your purpose as a missionary and serve others.
